In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import datetime
import os 

import sys
sys.path.append("C:/Users/tyrab/tyler_python/interview_prep/")
sys.path.append("C:/Users/tyrab/tyler_python/futures_trading/")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
!pip install --upgrade --user snowflake_connector_python

Requirement already up-to-date: snowflake_connector_python in c:\users\tyrab\appdata\roaming\python\python37\site-packages (2.7.6)
  Attempting uninstall: pyOpenSSL
    Found existing installation: pyOpenSSL 22.0.0
    Uninstalling pyOpenSSL-22.0.0:
      Successfully uninstalled pyOpenSSL-22.0.0


In [8]:
!pip install --user pyarrow==6.0.1

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import snowflake.connector

In [3]:
import pyarrow

In [4]:
pyarrow.__version__

'6.0.1'

In [4]:
con = snowflake.connector.connect(
    user='tyrabe486',
    password='TE18_Hershey',
    account='bc44100.us-east-2.aws'
)

In [5]:
# Create cursor
cur = con.cursor()

In [6]:
# Execute SQL statement
cur.execute("select current_date;")

In [7]:
print(cur.fetchone()[0])

2022-04-17


In [8]:
#Create data warehouse 
con.cursor().execute("CREATE WAREHOUSE IF NOT EXISTS trading_data")
con.cursor().execute("CREATE DATABASE IF NOT EXISTS futures_data")
con.cursor().execute("USE DATABASE futures_data")
con.cursor().execute("CREATE SCHEMA IF NOT EXISTS ES")

In [9]:
# con.cursor().execute("USE SCHEMA futures_data.ES")

In [26]:
from dataframe_functions.input_preprocessing import load_data, filter_open
from etl.processing.create_price_data import PriceData

In [58]:
es_data = load_data('es', base_path='C:/Users/tyrab/tyler_python/futures_trading/etl/input_data/')

In [59]:
es_all = PriceData(es_data)

In [ ]:
es_all.

In [60]:
daily_data = es_all.df_daily

In [61]:
daily_data['DATE_COL'] =daily_data.timestamp.dt.date

In [62]:
daily_data.columns = daily_data.columns.str.upper()

In [63]:
daily_data = daily_data[['DATE_COL', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME']]

In [64]:
daily_data.head()

,DATE_COL,OPEN,HIGH,LOW,CLOSE,VOLUME
0,2015-01-02,2061.25,2066.50,2038.75,2046.25,1162587
3,2015-01-05,2038.25,2040.50,2009.50,2013.75,1735654
4,2015-01-06,2017.50,2023.50,1984.25,1993.50,2026837
5,2015-01-07,2010.25,2023.75,2004.75,2018.50,1436722
6,2015-01-08,2036.50,2058.50,2036.00,2055.75,1242665


In [65]:
daily_data.dtypes

DATE_COL     object
OPEN        float64
HIGH        float64
LOW         float64
CLOSE       float64
VOLUME        int64
dtype: object

In [31]:
from snowflake.connector.pandas_tools import write_pandas

In [91]:
#Create the SQL statement to create or replace the table
dataframe = daily_data

create_tbl_statement = "CREATE OR REPLACE TABLE " + 'ES_DAILY' + " (\n"

# Loop through each column finding the datatype and adding it to the statement

for column in dataframe.columns:
    if (
    dataframe[column].dtype.name == "int"
    or dataframe[column].dtype.name == "int64"
    ):
        create_tbl_statement = create_tbl_statement + column + " INT"
    elif dataframe[column].dtype.name == "object":
        create_tbl_statement = create_tbl_statement + column + " DATE"
    elif dataframe[column].dtype.name == "datetime64[ns]":
        create_tbl_statement = create_tbl_statement + column + " TIMESTAMP"
    elif dataframe[column].dtype.name == "float64":
        create_tbl_statement = create_tbl_statement + column + " FLOAT"
#     elif dataframe[column].dtype.name == "bool":
#         create_tbl_statement = create_tbl_statement + column + " boolean"
    else:
        create_tbl_statement = create_tbl_statement + column + " varchar(16777216)"

    # If column is not last column, add comma, else end sql-query
    if dataframe[column].name != dataframe.columns[-1]:
        create_tbl_statement = create_tbl_statement + ",\n"
    else:
        create_tbl_statement = create_tbl_statement + ")"

In [92]:
create_tbl_statement

'CREATE OR REPLACE TABLE ES_DAILY (\nDATE_COL DATE,\nOPEN FLOAT,\nHIGH FLOAT,\nLOW FLOAT,\nCLOSE FLOAT,\nVOLUME INT)'

In [93]:
con.cursor().execute(create_tbl_statement)

In [94]:
# Write Data to Snowflake.
success, nchunks, nrows, _ = write_pandas(con, daily_data, table_name='ES_DAILY')

In [95]:
cur = con.cursor()
cur.execute('select * from ES_DAILY')

In [96]:
cur.fetch_pandas_all()

,DATE_COL,OPEN,HIGH,LOW,CLOSE,VOLUME
0,2015-01-02,2061.25,2066.50,2038.75,2046.25,1162587
1,2015-01-05,2038.25,2040.50,2009.50,2013.75,1735654
2,2015-01-06,2017.50,2023.50,1984.25,1993.50,2026837
3,2015-01-07,2010.25,2023.75,2004.75,2018.50,1436722
4,2015-01-08,2036.50,2058.50,2036.00,2055.75,1242665
...,...,...,...,...,...,...
510,2017-12-22,2688.25,2689.50,2682.25,2688.50,467637
511,2017-12-26,2685.50,2688.25,2684.25,2687.25,302110
512,2017-12-27,2688.25,2691.00,2681.50,2685.25,503337
513,2017-12-28,2688.25,2689.00,2683.00,2686.00,419437


In [97]:
con.close()